# Inheritance and Polymorphism
Inheritance and polymorphism are the core concepts of OOP that enable efficient and consistent code reuse. Learn how to inherit from a class, customize and redefine methods, and review the differences between class-level data and instance-level data.

## Instance and class data <a name="one"></a>

### Core Principles of OOP
#### Inheritance:
- Extending functionality of existing code
#### Polymorphism:
- Creating a unified interface
#### Encapsulation:
- Building of data and methods

### Instance-level data
        class Employee:
            def __init__(self, name, salary):
                self.name = name
                self.salary = salary

        
        emp1 = Employee("Teo Mille" 50000)
        emp2 = Employee("Marta Popov", 65000)

- `name`, `salary` are *instance attributes*
- `self` binds to an instance

### Class-level data
- But what if you needed to store some data that is shared among all instances of a class?
    - E.g. What if you wanted to introduce a minimal salary across the entire org?
- Class-level data is data shared among all instances of a class
- Define *class attributes* in the body of `class`,

        class MyClass:
            # Define a class attribute
            CLASS_ATTR_NAME = attr_value


- Class attribute will serve as a "global variable" within the class
- `MIN_SALARY` is shared among all instances,

        class Employee:
            # Define a class attribute
            MIN_SALARY = 30000           # <--- no self.
            def __init__(self, name, salary):
                self.name = name
                # Use class name to access class attribute
                if salary >= Employee.MIN_SALARY:
                    self.salary = salary
                else:
                    self.salary = Employee.MIN_SALARY


- Don't use `self` to *define* class attribute
- Instead use `ClassName.ATTR_NAME` to *access* the class attribute value
- We can access it like any other attribute from an object instance

        emp1 = Employee("TBD", 40000)
        emp2 = Employee("TBD", 60000)
        # both will return 30000, and will be the same across all instances
        print(emp1.MIN_SALARY)
        print(emp2.MIN_SALARY)


### Why use class attributes?
**Global constants related to the class**  
- minimal/maximal values for attributes
- commonly used values and constance, e.g. `pi` for a `Circle` class
- ...

### Class methods
- Methods are already "shared": same code for every instance
    - Only difference is the data that is fed into it
- Class methods can use instance-level data
- To define a class method, you start with a class method decorator, followed by a method definition

        class MyClass:

            @classmethod                               # <--- use decorator to declare a class method
            def my_awesome_method(cls, args, ...):     # <--- cls argument refers to the class
                # Do stuff here
                # Can't use any instance attributes :(


- To call a class method, use a class-dot method syntax rather than object-dot method syntax,

        MyClass.my_awesome_method(args...)

- So why should we ever need class methods?

### Alternative constructors
- Main use case is alternative constructors
- A class can have only one `__init__` method, but there might be multiple ways to initialize an object
- E.g. We may want to create an `Employee` object from data stored in a file
    - We can't use a method, because that would require an instance and there isn't one yet
    - In the following, we introduce class method `from_file` that accepts a file name, reads the first line that presumably contains the name of the employee, and returns an object instance
    - In the return statement we use the `cls` variable, remember that this refers to the class, so this line will call the `__init__(...)` constructor just like `Employee()` would outside the class definition

            class Employee:
                MIN_SALARY = 30000
                def __init__(self, name, salary=30000):
                    self.name = name
                    if salary >= Employee.MIN_SALARY:
                        self.salary = salary
                    else:
                        self.salary = Employee.MIN_SALARY

                @classmethod
                def from_file(cls, filename):
                    with open(filename, "r") as f:
                        name = f.readline()
                    return cls(name)
                

            # Create an employee without calling Employee()
            emp = Employee.from_file("emloyee_data.txt")
            type(emp)


## Class inheritance <a name="two"></a>

### Code resuse
- OOP is essentially about code reuse
1. Someone has already done it
    - Modules (like `numpy` or `pandas`) are great for fixed functionality
    - OOP is great for customizing functionality
2. DRY: Don't Repeat Yourself
    - You may resuse your code over and over

### Inheritance
- Class inheritance is mechanism by which we can define a new class that gets all the functionality of another class plus maybe something extra without re-implementing the code
- New class functionality = Old class functionality + extra

![Inheritance](imgs/inheritance.png)

### Implementing class inheritance

        class MyChild(MyParent):
            # Do stuff here


- `MyParent`: class whose functionality is being extended/inherited
- `MyChild`: class that will inherit the functionality and add more

        class BankAccount:
            def __init__(self, balance):
                self.balance = balance

            def withdraw(self, amount):
                self.balance -= amount

        # Empty class inherited from BankAccount
        class SavingsAccount(BankAccount):
            pass


### Child class has all of the parent data

        # Constructor inherited from BankAccount
        savings_acct = SavingsAccount(1000)

        # Attribute inherited from BankAccount
        print(savings_acct.balance)

        # Method inherited from BankAccount
        savings_acct.withdraw(300)


### Inheritance: "is-a" relationship
*A `SavingsAccount` is a `BankAccount`*  
(possible with special features)

        savings_acct = SavingsAccount(1000)
        isinstance(savings_acct, SavingsAccount)      # returns True
        isinstance(savings_acct, BankAccount)         # returns True

        acct = BankAccount(500)
        isinstance(acct, SavingsAccount)              # returns False
        isinstance(acct, BankAccount)                 # returns True 


## Customizing functionality via inheritance <a name="three"></a>

### What we have so far

        class BankAccount:
            def __init__(self, balance):
                self.balance = balance

            def withdraw(self, amount):
                self.balance -= amount

        # Empty class inherited from BankAccount
        class SavingsAccount(BankAccount):
            pass

`SavingsAccount` does not have any functionality that `BankAccount` doesn't.

### Customizing constructors

        class SavingsAccount(BankAccount):

            # Constructor specifically for SavingsAccount with an additional parameter
            def __init__(self, balance, interest_rate):
                # Call the parent constructor using ClassName.__init__()
                BankAccount.__init__(self, balance)     # <--- self is a SavingsAccount but also a BankAccount
                # Add more functionality
                self.interest_rate = interest_rate


- Can run constructor of the parent class first by `Parent.__init__(self, args...)`
- Add more functionality
- Don't *have* to call the parent constructors, you can use new code entirely

### Create object with a customized constructor

        acct = SavingsAccount(1000, 0.03)
        print(acct.interest_rate)

### Adding functionality
- Add methods as usual
- Can use the data from both the parent and the child class

        class SavingsAccount(BankAccount):

            def __init__(self, balance, interest_rate):
                BankAccount.__init__(self, balance)    
                self.interest_rate = interest_rate

            # New functionality
            def compute_interest(self, n_periods = 1):
                return self.balance * ( ( 1 + self.interest_rate) ** n_periods - 1)

### Customizing functionality

        class CheckingsAccount(BankAccount):
            def __init__(self, balance, limit):
                BankAccount.__init__(self, content)    
                self.limit = limit

            def deposit(self, amount):
                self.balance += amount

            def withdraw(self, amount, fee=0):
                if fee <= self.limit:
                    BankAccount.withdraw(self, amount - fee)
                else:
                    BankAccount.withdraw(self, amount - self.limit)

    
- Can change the signature (add parameters)
- Use `Parent.method(self, args...)` to call a method from the parent class


        check_acct = CheckingAccount(1000, 25)

        # Will call withdraw from CheckingAccount
        check_acct.withdraw(200)

        bank_acct = BankAccount(1000)

        # Will call withdraw from BankAccount
        bank.acct.withdraw(200)

- Another difference for `CheckingAccount` instance, we could call the method with 2 parameters


        # Will call withdraw from CheckingAccount
        check_acct.withdraw(200, fee=15)

        # Will produce an error
        bank_acct.withdraw(200, fee=15)